# Pendientes

Nada

# Gpu

In [ ]:
!nvidia-smi

Thu Sep 25 15:04:13 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   44C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

# Set_Up

In [ ]:

root_data = f'/content/drive/MyDrive/Course Folder/Forex/XAUUSD/'
print(root_data)

direction = 'Short'
direction_number = -1

symbol = 'XAUUSD'
strategy = 'Kalman'
time_frame = 'M5'

trade_evolution = 'st_Max'
result_field = 'st_PnL'


/content/drive/MyDrive/Course Folder/Forex/XAUUSD/


# Libraries

In [ ]:
!pip install ta-lib
import talib as ta
print(ta.__version__)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 100.3 MB/s eta 0:00:00
0.6.7


In [ ]:
import numpy as np
import pandas as pd
import os
import joblib
import math
import time

from itertools import combinations, product

from tqdm.auto import tqdm

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFECV
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import roc_auc_score, r2_score
from sklearn.model_selection import TimeSeriesSplit
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

from scipy.cluster.hierarchy import linkage, fcluster
from scipy.spatial.distance import squareform

from xgboost import XGBClassifier, XGBRegressor

import tensorflow as tf

import sys
sys.path.append("..")

from __future__ import annotations
from typing import Tuple, List, Optional, Dict, Any, Union

from xgboost import XGBClassifier, XGBRegressor
from sklearn.metrics import (roc_auc_score, f1_score, accuracy_score, log_loss, r2_score)

from sklearn.preprocessing import LabelEncoder

import warnings
warnings.filterwarnings('ignore')

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use("seaborn-v0_8-darkgrid")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Calculate_Features



## Features

In [ ]:
def kalman_line(source, kalman_length: int, smooth: int):
    """
    Pine -> Python (solo 'kalman_line'), replicando la EMA de TradingView con
    *semilla SMA* (como ta.ema) sobre el núcleo Kalman kf_c.

    Parámetros
    ----------
    source : pd.Series o array-like de floats (precio crudo, sin diff/returns)
    kalman_length : int   (equivale a length_kal en Pine)
    smooth : int          (equivale a smooth_kal en Pine -> ta.ema(kf_c, smooth))

    Retorna
    -------
    Mismo tipo que `source`: pd.Series o np.ndarray con la línea Kalman suavizada.
    """
    import numpy as np
    import pandas as pd

    # normalizamos tipos
    is_series = hasattr(source, "index")
    idx = source.index if is_series else None
    x = np.asarray(source, dtype=np.float64)
    n = x.shape[0]
    if n == 0:
        return source

    # ---------- núcleo Kalman idéntico al Pine ----------
    sqrt_term   = np.sqrt((kalman_length / 10000.0) * 2.0)
    length_term = kalman_length / 10000.0

    kf_c   = np.empty(n, dtype=np.float64)
    velo_c = np.empty(n, dtype=np.float64)

    # bar 0 (nz(kf_c[1], source) y nz(velo_c[1], 0))
    kf_c[0] = x[0]
    velo_c[0] = 0.0

    for i in range(1, n):
        prev_kf = kf_c[i - 1]
        dk_c = x[i] - prev_kf
        smooth_c = prev_kf + dk_c * sqrt_term
        velo_c[i] = velo_c[i - 1] + length_term * dk_c
        kf_c[i] = smooth_c + velo_c[i]

    # ---------- EMA con semilla SMA (comportamiento ta.ema de TV) ----------
    L = int(max(1, smooth))
    alpha = 2.0 / (L + 1.0)
    ema = np.full(n, np.nan, dtype=np.float64)

    if n < L:
        # con pocas barras, igualamos al promedio simple disponible
        ema[-1] = np.nanmean(kf_c)
    else:
        # seed = SMA de las primeras L barras
        seed = np.mean(kf_c[:L])
        ema[L - 1] = seed
        for i in range(L, n):
            ema[i] = alpha * kf_c[i] + (1.0 - alpha) * ema[i - 1]

    return (pd.Series(ema, index=idx) if is_series else ema)


In [ ]:
def slope(src: pd.Series,
          length_kal: int,
          smooth_kal: int,
          slopeLen: int,
          offset: int) -> pd.DataFrame:

    n = len(src)
    kf_state = np.full(n, np.nan)
    kf_velo  = np.zeros(n)
    sqrt_factor = np.sqrt(length_kal / 10000.0 * 2.0)
    vel_factor  = length_kal / 10000.0

    for i in range(n):
        if i == 0:
            prev_state = src.iloc[0]
            prev_velo  = 0.0
        else:
            prev_state = kf_state[i-1] if not np.isnan(kf_state[i-1]) else src.iloc[i]
            prev_velo  = kf_velo[i-1]

        dk = src.iloc[i] - prev_state
        smooth = prev_state + dk * sqrt_factor
        kf_velo[i]  = prev_velo + vel_factor * dk
        kf_state[i] = smooth + kf_velo[i]

    # 2) EMA smoothing --------------------------------------------------
    kal = pd.Series(kf_state, index=src.index).ewm(span=smooth_kal, adjust=False).mean()

    # 3) Slope/divergence -----------------------------------------------
    validLen = max(slopeLen, 1)
    slope_div = kal.diff(validLen) / validLen
    slope_signal = (slope_div > slope_div.shift(1)).astype(int)

    # 4) Angle in degrees -----------------------------------------------
    price_change = kal - kal.shift(validLen)
    slope_angle = np.degrees(np.arctan(price_change))
    slope_angle_signal = (slope_angle > slope_angle.shift(1)).astype(int)

    # 5) Linear regression prediction ----------------------------------
    def _linreg(y):
        x = np.arange(len(y))
        m, b = np.polyfit(x, y, 1)
        return b + m * (len(y)-1)

    slope_lin_reg = kal.rolling(window=slopeLen).apply(_linreg, raw=False)
    slope_lin_reg = slope_lin_reg.shift(-offset)  # apply Pine-style offset
    slope_lin_reg_signal = (slope_lin_reg > slope_lin_reg.shift(1)).astype(int)

    # 6) Pack results ---------------------------------------------------
    return pd.DataFrame({
        'slope_div':            slope_div,
        'slope_signal':         slope_signal,
        'slope_angle':          slope_angle,
        'slope_angle_signal':   slope_angle_signal,
        'slope_lin_reg':        slope_lin_reg,
        'slope_lin_reg_signal': slope_lin_reg_signal
    })

In [ ]:
def build_slope_feature_block(stock_data: pd.DataFrame,
                                 kalman_periods: List[int],
                                 slope_lengths: List[int]) -> pd.DataFrame:
    """Assemble the complete slope feature block for the provided parameters.

    Parameters
    ----------
    stock_data : pd.DataFrame
        Input OHLCV data.
    kalman_periods : List[int]
        Window lengths used for the Kalman filter baseline.
    slope_lengths : List[int]
        Lookback windows used when computing the slope statistics.

    Returns
    -------
    pd.DataFrame
        A DataFrame containing all slope-based indicators, their diffs and pairwise combinations.
    """
    slope_features = pd.DataFrame(index=stock_data.index)
    slope_indicator_columns: Dict[str, List[str]] = {
        'slope_div': [],
        'slope_signal': [],
        'slope_angle': [],
        'slope_angle_signal': [],
        'slope_lin_reg': [],
        'slope_lin_reg_signal': []
    }

    slope_jobs = list(product(kalman_periods, slope_lengths))

    for period, s_len in tqdm(slope_jobs, desc="Slopes (kal_len × slope_len)"):
        df_s = slope(stock_data['Close'], length_kal=period, smooth_kal=3,
                     slopeLen=s_len, offset=-1)

        column_mapping = {
            'slope_div':            f'slope_div_{period}_{s_len}',
            'slope_signal':         f'slope_signal_{period}_{s_len}',
            'slope_angle':          f'slope_angle_{period}_{s_len}',
            'slope_angle_signal':   f'slope_angle_signal_{period}_{s_len}',
            'slope_lin_reg':        f'slope_lin_reg_{period}_{s_len}',
            'slope_lin_reg_signal': f'slope_lin_reg_signal_{period}_{s_len}'
        }

        for indicator, col_name in column_mapping.items():
            series = df_s[indicator]
            slope_features[col_name] = series
            slope_features[f'{col_name}_diff'] = series.diff()
            slope_indicator_columns[indicator].append(col_name)

    def _unique_pairwise(columns: List[str]) -> List[Tuple[str, str]]:
        unique_columns = list(dict.fromkeys(columns))
        return list(combinations(unique_columns, 2))

    for indicator, cols in slope_indicator_columns.items():
        pairs = _unique_pairwise(cols)
        for c1, c2 in pairs:
            slope_features[f'{c1} - {c2}'] = slope_features[c1] - slope_features[c2]

    return slope_features


In [ ]:
def create_features(
    stock_data: pd.DataFrame,
    return_components: bool = False
) -> Union[pd.DataFrame, Tuple[pd.DataFrame, Dict[str, pd.DataFrame]]]:

    periods        = [3, 7, 14]
    kalman_periods = [300, 600, 900]
    slope_length   = [3, 6, 9]

    features = pd.DataFrame(index=stock_data.index)
    component_frames: Dict[str, pd.DataFrame] = {}

    basic_features = basic_indicators(stock_data)
    component_frames['Basic_Indicators'] = basic_features
    features = features.join(basic_features)

    volume_features = volume_indicators(stock_data)
    component_frames['Volume_Indicators'] = volume_features
    features = features.join(volume_features)

    ema_features, _ = add_ema_feature_block(stock_data, ema_lengths)
    component_frames['EMA_Features'] = ema_features
    features = features.join(ema_features)

    def _unique_pairwise(columns: List[str]) -> List[Tuple[str, str]]:
        """Return ordered unique column pairs without self-pairings."""
        unique_columns = list(dict.fromkeys(columns))
        return list(combinations(unique_columns, 2))

    # ───────────────────────── Kalman y derivados ───────────────────────
    t0 = time.time()
    kal_cols = []
    kalman_900_series: Optional[pd.Series] = None
    for period in tqdm(kalman_periods, desc="Kalman & Derivatives"):
        kal = pd.Series(
            kalman_line(stock_data['Close'], kalman_length=period, smooth=3),
            index=stock_data.index
        )
        kname = f'Kal_{period}'
        kal_cols.append(kname)

        features[kname]                           = kal
        if period == 900:
            kalman_900_series = kal
        features[f'Close_Kal_{period}']           = stock_data['Close'] - kal
        features[f'Kal_change_{period}']          = kal.diff(1)
        features[f'Kal_prev_minus_now_{period}']  = kal.shift(1) - kal
        features[f'Kal_prev2_minus_now_{period}'] = kal.shift(2) - kal
        features[f'Kal_change2_{period}']         = kal.diff(2)

    kal_pairs = _unique_pairwise(kal_cols)
    for c1, c2 in tqdm(kal_pairs, desc="Kalman pairwise"):
        features[f'{c1}_minus_{c2}'] = features[c1] - features[c2]
    tqdm.write(f"[Timing] Kalman block: {time.time()-t0:.2f}s")

    # ───────────────────────── RSI (Close & Kalman_900) ────────────────────
    t0 = time.time()
    rsi_sources: Dict[str, pd.Series] = {'Close': stock_data['Close']}
    if kalman_900_series is not None:
        rsi_sources['Kalman_900'] = kalman_900_series

    for source_name, series in rsi_sources.items():
        rsi_cols: List[str] = []
        for period in tqdm(periods, desc=f"RSI ({source_name})", leave=False):
            if source_name == 'Close':
                col_name = f'RSI_{period}'
            else:
                col_name = f'RSI_{source_name}_{period}'
            features[col_name] = ta.RSI(series, timeperiod=period)
            rsi_cols.append(col_name)

        for col in tqdm(rsi_cols, desc=f"RSI ({source_name}) diffs", leave=False):
            features[f'{col}_diff'] = features[col].diff()

        for col1, col2 in tqdm(_unique_pairwise(rsi_cols), desc=f"RSI ({source_name}) pairwise", leave=False):
            features[f'{col1} - {col2}'] = features[col1] - features[col2]

    tqdm.write(f"[Timing] RSI block: {time.time()-t0:.2f}s")

    # ───────────────────────── Slopes ───────────────────────────────────
    t0 = time.time()
    slope_features = build_slope_feature_block(stock_data, kalman_periods, slope_length)
    component_frames['Slope'] = slope_features
    features = features.join(slope_features)
    tqdm.write(f"[Timing] Slopes block: {time.time()-t0:.2f}s")

    if return_components:
        component_frames['Create_Features'] = features.copy()
        return features, component_frames

    return features

In [ ]:
def scale_feature_block(features: pd.DataFrame, window: int = 200) -> pd.DataFrame:
    """Scale features using a rolling window standardization.

    Parameters
    ----------
    features : pd.DataFrame
        Feature block to scale.
    window : int, optional
        Rolling window size, by default 200.

    Returns
    -------
    pd.DataFrame
        Scaled feature block preserving the original index.
    """
    if features.empty:
        return features.copy()

    scaled = features.copy()
    rolling = scaled.rolling(window)
    scaled = (scaled - rolling.mean()) / rolling.std()
    return scaled


## 5_min

In [ ]:
# Read the CSV file
df_5min = pd.read_csv(root_data + 'Data/'+symbol+'_M5.csv', index_col=0)
df_5min.index = pd.to_datetime(df_5min.index)
#df_5min = df_5min.iloc[-1000000:,]

print('Min_Date : ', df_5min.index.min())
print('Min_Date : ', df_5min.index.max())
print('Number_Rows = ',len(df_5min.index))
print('\n')

df_5min.tail()

Min_Date :  2019-01-02 01:00:00
Min_Date :  2025-07-25 23:55:00
Number_Rows =  465718




,Open,High,Low,Close,Volume,Spread
Date,,,,,,
2025-07-25 23:35:00,3338.59,3339.14,3338.04,3338.81,197,5
2025-07-25 23:40:00,3338.81,3338.92,3337.56,3338.69,194,5
2025-07-25 23:45:00,3338.69,3338.69,3336.64,3336.73,195,40
2025-07-25 23:50:00,3336.73,3336.86,3336.12,3336.52,192,40
2025-07-25 23:55:00,3336.52,3336.88,3336.44,3336.80,82,40


**Features**

In [ ]:
start_time = time.time()

features_5min_raw, components_5min = create_features(df_5min, return_components=True)
features_5min_raw = features_5min_raw.dropna()
valid_index_5min = features_5min_raw.index

# Align every component to the valid index coming from the full feature set
m5_raw_blocks: Dict[str, pd.DataFrame] = {name: frame.loc[valid_index_5min].copy()
                                           for name, frame in components_5min.items()}
m5_raw_blocks['Create_Features'] = features_5min_raw

m5_scaled_blocks: Dict[str, pd.DataFrame] = {}
m5_raw_paths: Dict[str, str] = {}
m5_scaled_paths: Dict[str, str] = {}

for name, frame in m5_raw_blocks.items():
    filename_raw = f"{symbol}_M5_{name}_Raw_Features.csv" if name != 'Create_Features' else f"{symbol}_M5_Raw_Features.csv"
    raw_path = os.path.join(root_data, 'Results', filename_raw)
    frame.to_csv(raw_path)
    m5_raw_paths[name] = raw_path

    scaled_frame = scale_feature_block(frame)
    m5_scaled_blocks[name] = scaled_frame
    filename_scale = f"{symbol}_M5_{name}_Scale_Features.csv" if name != 'Create_Features' else f"{symbol}_M5_Scale_Features.csv"
    scale_path = os.path.join(root_data, 'Results', filename_scale)
    scaled_frame.to_csv(scale_path)
    m5_scaled_paths[name] = scale_path

execution_time = time.time() - start_time

print(f"Number of features are: {features_5min_raw.shape[1]}")
print(features_5min_raw.shape)
print(f"Execution time: {execution_time:.2f} seconds")
print(f"Saved {len(m5_raw_blocks) * 2} DataFrames for the M5 timeframe.")
features_5min_raw.tail(5)


In [ ]:
print("NaN counts per column (sorted):")
print(m5_raw_blocks['Create_Features'].isnull().sum().sort_values(ascending=False), '')

In [ ]:
print("Saved raw feature files:")
for name, path in m5_raw_paths.items():
    print(f" - {name}: {path}")

**Scale_features**

In [ ]:
print("Saved scaled feature files:")
for name, path in m5_scaled_paths.items():
    print(f" - {name}: {path}")


In [ ]:
m5_scaled_blocks['Create_Features'].head()

In [ ]:
print("NaN counts per column (scaled, sorted):")
print(m5_scaled_blocks['Create_Features'].isnull().sum().sort_values(ascending=False), '')

In [ ]:
print("M5 feature blocks:", list(m5_raw_blocks.keys()))

**Plots**

In [ ]:
import matplotlib.pyplot as plt

last_200_features_15min = m15_scaled_blocks['Create_Features'].tail(200)
last_200_df_15min = df_15min.tail(200)

indicators_to_plot = ['Close','15min_Kal_300','15min_Kal_600']

# Separate Close from other indicators
close_to_plot = 'Close' if 'Close' in indicators_to_plot else None
other_indicators_to_plot = [col for col in indicators_to_plot if col != 'Close' and col in last_200_features_15min.columns]

num_plots = len(other_indicators_to_plot) + (1 if close_to_plot else 0)

# Plotting
fig, axes = plt.subplots(nrows=num_plots, ncols=1, figsize=(15, 2.5 * num_plots), sharex=True)

# Ensure axes is an array even if only one plot
if not isinstance(axes, np.ndarray):
    axes = np.array([axes])

current_plot_index = 0

# Plot Close price if requested
if close_to_plot:
    axes[current_plot_index].plot(last_200_df_15min.index, last_200_df_15min['Close'])
    axes[current_plot_index].set_title('Close Price')
    axes[current_plot_index].grid(True)
    current_plot_index += 1

# Plot each selected indicator (excluding 'Close')
for col in other_indicators_to_plot:
    axes[current_plot_index].plot(last_200_features_15min.index, last_200_features_15min[col])
    axes[current_plot_index].set_title(col)
    axes[current_plot_index].grid(True)
    current_plot_index += 1

#plt.tight_layout()
#plt.show()


# Feature Importance

## Labels

In [ ]:
lab = pd.read_csv(root_data + 'Results/'+symbol+'_'+strategy+'_'+time_frame+'_Strategy_Gen_Labels.csv', index_col=0)
lab['Date'] = pd.to_datetime(lab['Date'])

print('Min_Date    : ',lab['Date'].min())
print('Min_Date    : ',lab['Date'].max(),'\n')
print('Number_Rows : ',lab.shape,'\n')
print('Columns     : ',lab.columns)

lab['Open_Trade'].value_counts()

Min_Date    :  2019-01-02 01:00:00
Min_Date    :  2025-07-25 23:55:00 

Number_Rows :  (465718, 29) 

Columns     :  Index(['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Spread', 'ATR',
       'kal_1', 'kal_2', 'kal_3', 'kal_4', 'Open_Trade', 'Close_Trade',
       'Entry_Date', 'Type', 'Trade_Number', 'st_Exit_Date', 'trade type',
       'st_Duration', 'st_row_PnL_close', 'st_row_PnL_high', 'st_row_PnL_Low',
       'st_row_PnL_low', 'st_Max', 'st_Min', 'st_PnL', 'st_atr_PnL',
       'st_atr_max_PnL'],
      dtype='object')


,count
Open_Trade,
1.0,33915
-1.0,33858


In [ ]:
#analyse_column = 'st_atr_max_PnL'
analyse_column = 'st_Max'

st_max_0  = lab.loc[((lab['Open_Trade']==1) |(lab['Open_Trade']==-1)),:]['Open_Trade'].count()
st_max_1  = lab.loc[((lab['Open_Trade']==1) |(lab['Open_Trade']==-1)),analyse_column].mean()
st_max_2  = lab.loc[((lab['Open_Trade']==1) |(lab['Open_Trade']==-1)) & (lab['st_atr_max_PnL'] >= 1.93),analyse_column].sum()
st_max_25 = lab.loc[((lab['Open_Trade']==1) |(lab['Open_Trade']==-1)) & (lab['st_atr_max_PnL'] <= 1.93),analyse_column].sum()


st_max_3 = lab.loc[(((lab['Open_Trade']==1) |(lab['Open_Trade']==-1)) & (lab['st_atr_max_PnL'] <= 0)),analyse_column].sum()

st_max_4 = lab.loc[(((lab['Open_Trade']==1) |(lab['Open_Trade']==-1)) &
                   (lab['st_atr_max_PnL'] > 0.7) & (lab['st_atr_max_PnL'] <= 1)),analyse_column].sum()

st_max_5 = lab.loc[(((lab['Open_Trade']==1) |(lab['Open_Trade']==-1)) &
                   (lab['st_atr_max_PnL'] > 1) &
                   (lab['st_atr_max_PnL'] <= 1.5)),analyse_column].sum()

st_max_6 = lab.loc[(((lab['Open_Trade']==1) |(lab['Open_Trade']==-1)) &
                   (lab['st_atr_max_PnL'] > 1.5) &
                   (lab['st_atr_max_PnL'] <= 2)),analyse_column].sum()

st_max_7 = lab.loc[(((lab['Open_Trade']==1) |(lab['Open_Trade']==-1)) &
                   (lab['st_atr_max_PnL'] > 2)),analyse_column].sum()


print(f'Total_Trades = {st_max_0:,.0f}\n')
print(f'Mean st_atr_max_PnL = {st_max_1:,.2f}\n')
print(f'Above_Mean = {st_max_2:,.2f}')
print(f'Below_Mean = {st_max_25:,.2f}\n')

print(f'<= 0.5 = {st_max_3:,.2f}')
print(f'> 0.5 & <= 1 = {st_max_4:,.2f}')
print(f'> 1 & <= 1.5 = {st_max_5:,.2f}')
print(f'> 1.5 & <= 2 = {st_max_6:,.2f}')
print(f'> 2 = {st_max_7:,.2f}')

Total_Trades = 67,773

Mean st_atr_max_PnL = 238.39

Above_Mean = 11,463,914.00
Below_Mean = 4,691,967.00

<= 0.5 = 33.00
> 0.5 & <= 1 = 801,960.00
> 1 & <= 1.5 = 1,411,119.00
> 1.5 & <= 2 = 1,400,438.00
> 2 = 11,277,394.00


In [ ]:
analyse_column = 'st_atr_max_PnL'

st_max_0 = lab.loc[((lab['Open_Trade']==1) |(lab['Open_Trade']==-1)),:]['Open_Trade'].count()
st_max_1 = lab.loc[((lab['Open_Trade']==1) |(lab['Open_Trade']==-1)),analyse_column].mean()
st_max_2 = lab.loc[((lab['Open_Trade']==1) |(lab['Open_Trade']==-1)) & (lab['st_atr_max_PnL'] >= 1.93),analyse_column].count()
st_max_25 = lab.loc[((lab['Open_Trade']==1) |(lab['Open_Trade']==-1)) & (lab['st_atr_max_PnL'] <= 1.93),analyse_column].count()

st_max_3 = lab.loc[(((lab['Open_Trade']==1) |(lab['Open_Trade']==-1)) &
                    (lab['st_atr_max_PnL'] <= 0.5)),analyse_column].count()

st_max_4 = lab.loc[(((lab['Open_Trade']==1) |(lab['Open_Trade']==-1)) &
                     (lab['st_atr_max_PnL'] >= 0.5) & (lab['st_atr_max_PnL'] <= 1)),analyse_column].count()

st_max_5 = lab.loc[((lab['Open_Trade']==1) |(lab['Open_Trade']==-1)) &
                   (lab['st_atr_max_PnL'] >= 1), analyse_column].count()

print(f'Total_Trades = {st_max_0:,.0f}\n')
print(f'Mean st_atr_max_PnL = {st_max_1:,.2f}\n')
print(f'Above_Mean = {st_max_2:,.2f}')
print(f'Below_Mean = {st_max_25:,.2f}\n')

print(f'<= 0.5 = {st_max_3:,.2f}')
print(f'> 0.5 & <= 1 = {st_max_4:,.2f}')
print(f'> 1 = {st_max_5:,.2f}')


Total_Trades = 67,773

Mean st_atr_max_PnL = 1.89

Above_Mean = 20,857.00
Below_Mean = 46,913.00

<= 0.5 = 20,431.00
> 0.5 & <= 1 = 13,116.00
> 1 = 34,223.00


In [ ]:
# --- Parámetros / campos
result_field = 'st_atr_max_PnL'

#valid = (
#    (lab['Type'] == direction) &
#    (lab['Open_Trade'].isin([1, -1])) &
#    (lab[result_field].notna()))

valid = (
    (lab['Open_Trade'].isin([1, -1])) &
    (lab[result_field].notna()))


# --- Etiquetado en la columna "label" con valores 4/5/6
lab['label'] = np.nan
lab.loc[valid & (lab[result_field] <= 1), 'label'] = 0
lab.loc[valid & (lab[result_field] >= 1), 'label'] = 1


# --- Mantener solo filas válidas y con label
lab = lab.loc[valid & lab['label'].notna()].copy()
lab['label'] = lab['label'].astype('int8')

# --- Ver distribución de labels 4/5/6
print('\nValue counts de label 4/5/6:')
print(lab['label'].value_counts(dropna=False).sort_index())



Value counts de label 4/5/6:
label
0    33547
1    34223
Name: count, dtype: int64


## Features

In [ ]:
raw_feat_5min = pd.read_csv(root_data+'Results/'+symbol+'_M5_Raw_Features.csv')
raw_feat_5min["Date"] = pd.to_datetime(raw_feat_5min["Date"])
print(raw_feat_5min.shape)
#raw_feat_5min.head(5)

(465518, 455)


In [ ]:
scale_feat_5min = pd.read_csv(root_data+'Results/'+symbol+'_M5_Scale_Features.csv')
#scale_feat_5min = scale_feat_5min.drop('Unnamed: 0', axis=1)
scale_feat_5min["Date"] = pd.to_datetime(scale_feat_5min["Date"])
print(scale_feat_5min.shape)
#scale_feat_5min.head(5)

(465518, 455)


## Merge

In [ ]:
data_type = 'Scale'

In [ ]:
def prepare_feature_block(frame: pd.DataFrame, prefix: str) -> pd.DataFrame:
    """Return a copy of the feature block with a consistent prefix."""
    block = frame.copy()
    if 'Unnamed: 0' in block.columns:
        block = block.drop(columns='Unnamed: 0')
    block = block.drop_duplicates(subset='Date').sort_values('Date')
    rename_map = {col: f"{prefix}{col}" for col in block.columns if col != 'Date'}
    block = block.rename(columns=rename_map)
    return block

feature_blocks = {
    'M5_raw': prepare_feature_block(raw_feat_5min, 'M5_raw_'),
    'M5_scale': prepare_feature_block(scale_feat_5min, 'M5_scale_'),
    'M10_raw': prepare_feature_block(raw_feat_10min, 'M10_raw_'),
    'M10_scale': prepare_feature_block(scale_feat_10min, 'M10_scale_'),
    'M15_raw': prepare_feature_block(raw_feat_15min, 'M15_raw_'),
    'M15_scale': prepare_feature_block(scale_feat_15min, 'M15_scale_'),
}

feature_df = lab[['Date']].drop_duplicates().copy()

print('Feature blocks merged:')
for name, block in feature_blocks.items():
    feature_df = feature_df.merge(block, on='Date', how='left')
    print(f" - {name}: {block.shape[1] - 1} columns")

feature_df = feature_df.sort_values('Date').set_index('Date').ffill().reset_index()

df = lab[['Date', 'label', 'Open_Trade']].merge(feature_df, on='Date', how='left')

cols = df.columns.tolist()
cols.remove('label')
cols.insert(1, 'label')
df = df[cols]

combined_feature_path = os.path.join(root_data, 'Results', f"{symbol}_{direction}_AllFeatures.csv")
df.to_csv(combined_feature_path, index=False)

print(f"Combined feature dataframe shape: {df.shape}")
print(f"Saved combined feature dataframe to: {combined_feature_path}")

Feature blocks merged:
 - M5_raw: 454 columns
 - M5_scale: 454 columns
 - M10_raw: 454 columns
 - M10_scale: 454 columns
 - M15_raw: 454 columns
 - M15_scale: 454 columns
Combined feature dataframe shape: (67770, 2727)
Saved combined feature dataframe to: /content/drive/MyDrive/Course Folder/Forex/XAUUSD/Results/XAUUSD_Short_AllFeatures.csv


In [ ]:
expected_prefixes = ['M5_raw_', 'M5_scale_', 'M10_raw_', 'M10_scale_', 'M15_raw_', 'M15_scale_']
for prefix in expected_prefixes:
    matching_cols = [col for col in df.columns if col.startswith(prefix)]
    print(f"{prefix}: {len(matching_cols)} columns")

duplicated_columns = df.columns[df.columns.duplicated()].tolist()
if duplicated_columns:
    print('Duplicated feature columns detected:', duplicated_columns)
else:
    print('No duplicated feature columns detected.')

M5_raw_: 454 columns
M5_scale_: 454 columns
M10_raw_: 454 columns
M10_scale_: 454 columns
M15_raw_: 454 columns
M15_scale_: 454 columns
No duplicated feature columns detected.


In [ ]:
print(df.shape,'\n')
print('Label_Counts : ',df.label.value_counts(),'\n')
print(list(df.columns), '\n')

# Add NaN count per column, sorted
print("NaN counts per column (sorted):")
print(df.isnull().sum().sort_values(ascending=False), '\n')

#df.head(5)

(67770, 2727) 

Label_Counts :  label
1    34223
0    33547
Name: count, dtype: int64 

['Date', 'label', 'Open_Trade', 'M5_raw_MACD_12_26_9', 'M5_raw_MACD_12_26_9_diff', 'M5_raw_MACD_signal_12_26_9', 'M5_raw_MACD_signal_12_26_9_diff', 'M5_raw_MACD_hist_12_26_9', 'M5_raw_MACD_hist_12_26_9_diff', 'M5_raw_RSI_basic_14', 'M5_raw_RSI_basic_14_diff', 'M5_raw_BB_upper_20', 'M5_raw_BB_upper_20_diff', 'M5_raw_BB_middle_20', 'M5_raw_BB_middle_20_diff', 'M5_raw_BB_lower_20', 'M5_raw_BB_lower_20_diff', 'M5_raw_ATR_14', 'M5_raw_ATR_14_diff', 'M5_raw_OBV_basic', 'M5_raw_OBV_basic_diff', 'M5_raw_VWAP_basic', 'M5_raw_VWAP_basic_diff', 'M5_raw_Momentum_10', 'M5_raw_Momentum_10_diff', 'M5_raw_ROC_10', 'M5_raw_ROC_10_diff', 'M5_raw_Stoch_K_14_3_3', 'M5_raw_Stoch_K_14_3_3_diff', 'M5_raw_Stoch_D_14_3_3', 'M5_raw_Stoch_D_14_3_3_diff', 'M5_raw_CCI_20', 'M5_raw_CCI_20_diff', 'M5_raw_MFI_basic_14', 'M5_raw_MFI_basic_14_diff', 'M5_raw_ADX_14', 'M5_raw_ADX_14_diff', 'M5_raw_WilliamsR_14', 'M5_raw_WilliamsR_14_d

In [ ]:
### Select features containing 'diff' or '-' in their names
#diff_features = df.filter(regex='diff|-')

### Create a new dataframe with the Date column, label and the selected features
#df_diff = pd.concat([df[['Date', 'label']], diff_features], axis=1)

#print("DataFrame with difference features:")
#print(df_diff.shape,'\n')
#print('Label_Counts : ',df_diff.label.value_counts(),'\n')
#print(list(df_diff.columns), '\n')

# Add NaN count per column, sorted
#print("NaN counts per column (sorted):")
#print(df_diff.isnull().sum().sort_values(ascending=False), '\n')

#df_diff.head(5)

DataFrame with difference features:
(67770, 1922) 

Label_Counts :  label
1    34223
0    33547
Name: count, dtype: int64 

['Date', 'label', 'M5_raw_MACD_12_26_9_diff', 'M5_raw_MACD_signal_12_26_9_diff', 'M5_raw_MACD_hist_12_26_9_diff', 'M5_raw_RSI_basic_14_diff', 'M5_raw_BB_upper_20_diff', 'M5_raw_BB_middle_20_diff', 'M5_raw_BB_lower_20_diff', 'M5_raw_ATR_14_diff', 'M5_raw_OBV_basic_diff', 'M5_raw_VWAP_basic_diff', 'M5_raw_Momentum_10_diff', 'M5_raw_ROC_10_diff', 'M5_raw_Stoch_K_14_3_3_diff', 'M5_raw_Stoch_D_14_3_3_diff', 'M5_raw_CCI_20_diff', 'M5_raw_MFI_basic_14_diff', 'M5_raw_ADX_14_diff', 'M5_raw_WilliamsR_14_diff', 'M5_raw_CMF_diff', 'M5_raw_TMF_diff', 'M5_raw_MFI_14_volume_diff', 'M5_raw_KO_diff', 'M5_raw_EFI_diff', 'M5_raw_EOM_diff', 'M5_raw_VPT_diff', 'M5_raw_NVI_diff', 'M5_raw_PVI_diff', 'M5_raw_VFI_diff', 'M5_raw_VWAP_diff', 'M5_raw_Market_Facilitation_Index_diff', 'M5_raw_AD_Line_diff', 'M5_raw_WAD_diff', 'M5_raw_EMA_15_diff', 'M5_raw_EMA_21_diff', 'M5_raw_EMA_50_diff', 'M

In [ ]:
### Align feature directions so that shorts mirror longs
#if 'Open_Trade' not in df.columns:
 #   raise KeyError("'Open_Trade' column is required in df to flip feature signs.")

### Attach the trade direction to the diff dataframe (kept for reference).
#df_diff = df_diff.merge(df[['Date', 'Open_Trade']], on='Date', how='left')

### Identify feature columns to flip (exclude identifiers/targets).
#feature_cols = [col for col in df_diff.columns if col not in ['Date', 'label', 'Open_Trade']]
#short_mask = df_diff['Open_Trade'] == -1

#if short_mask.any():
#    df_diff.loc[short_mask, feature_cols] = df_diff.loc[short_mask, feature_cols] * -1
#    print(f"Flipped {short_mask.sum()} rows with Open_Trade = -1.")
#else:
#    print("No rows with Open_Trade = -1 were found.")

### Reorder columns so Open_Trade stays next to the label for downstream steps.
#ordered_cols = ['Date', 'label', 'Open_Trade'] + [col for col in feature_cols]
#df_diff = df_diff[ordered_cols]


Flipped 33857 rows with Open_Trade = -1.


In [ ]:
#df = df_diff.copy()

## ML

In [ ]:
# ===================== 1. ENTRENAR Y OBTENER IMPORTANCIAS =====================
def compute_xgb_importance(
    X: pd.DataFrame,
    y: pd.Series,
    task: str = "classification",
    random_state: int = 42,
    **xgb_params: Any
) -> Tuple[pd.DataFrame, Any]:
    """
    Entrena un modelo XGBoost y devuelve:
      - imp_df: DataFrame con 'feature', 'importance' y 'cum_importance'.
      - model : modelo ya entrenado.

    Soporta:
      • Clasificación binaria o multiclase (detecta nº de clases).
      • Regresión (si task != 'classification').

    Parámetros
    ----------
    X : pd.DataFrame
        Matriz de características (sin la columna objetivo).
    y : pd.Series
        Etiquetas objetivo. Puede ser binaria (0/1) o multiclase (0..K-1).
    task : str, opcional
        "classification" (default) o "regression".
    random_state : int, opcional
        Semilla para reproducibilidad.
    **xgb_params : dict
        Parámetros adicionales para el estimador de XGBoost.

    Returns
    -------
    (imp_df, model)
        imp_df : DataFrame con importancias y su acumulado.
        model  : instancia entrenada de XGBClassifier / XGBRegressor.
    """
    default_params: Dict[str, Any] = dict(
        n_estimators=500,
        max_depth=6,
        learning_rate=0.05,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=random_state,
        n_jobs=-1,
        tree_method="hist",
    )
    default_params.update(xgb_params)

    if task == "classification":
        # Detectar nº de clases
        classes = np.unique(y)
        n_classes = len(classes)

        # XGBClassifier ajusta objetivo automáticamente, pero lo explicitamos:
        if n_classes > 2:
            default_params.setdefault("objective", "multi:softprob")
            default_params.setdefault("num_class", n_classes)
            eval_metric = "mlogloss"
        else:
            default_params.setdefault("objective", "binary:logistic")
            eval_metric = "logloss"

        model = XGBClassifier(eval_metric=eval_metric, **default_params)

    else:
        model = XGBRegressor(**default_params)

    model.fit(X, y)

    imp_df = (
        pd.DataFrame({
            "feature": X.columns,
            "importance": model.feature_importances_
        })
        .sort_values("importance", ascending=False)
        .reset_index(drop=True)
    )
    total_imp = imp_df["importance"].sum()
    if total_imp == 0:
        # Evitar división por cero si el modelo devuelve todo cero (raro, pero posible)
        imp_df["cum_importance"] = 0.0
    else:
        imp_df["cum_importance"] = imp_df["importance"].cumsum() / total_imp

    return imp_df, model

In [ ]:
# ===================== 2. SELECCIÓN DE FEATURES =====================
def select_features_with_importance(
    X: pd.DataFrame,
    imp_df: pd.DataFrame,
    top_n: Optional[int] = None,
    threshold: Optional[str | float] = None,
    cum_threshold: Optional[float] = 0.8
) -> Tuple[pd.DataFrame, List[str]]:
    """
    Selección flexible de variables a partir de importancias de XGBoost.

    Reglas:
      - Si top_n no es None           => usa el top_n.
      - Else si cum_threshold no None => usa importancia acumulada (p.ej. 0.8 = 80%).
      - Else usa threshold ('median', 'mean' o valor numérico).

    Devuelve (X_reducido, lista_de_features).

    Parámetros
    ----------
    X : pd.DataFrame
        Matriz de características original.
    imp_df : pd.DataFrame
        DataFrame devuelto por compute_xgb_importance.
    top_n : int | None
        Número fijo de variables a conservar.
    threshold : str | float | None
        Umbral de importancia. Si str, usar 'median' o 'mean'.
    cum_threshold : float | None
        Porcentaje acumulado de importancia (0-1). Si None, se ignora.

    Returns
    -------
    (X_sel, keep)
        X_sel : subset de X con columnas seleccionadas.
        keep  : lista de nombres de columnas seleccionadas.
    """
    if top_n is not None:
        keep = imp_df.head(top_n)["feature"].tolist()

    elif cum_threshold is not None:
        keep_mask = imp_df["cum_importance"] <= float(cum_threshold)
        keep = imp_df.loc[keep_mask, "feature"].tolist()
        # asegurar que haya al menos una más para no quedarnos exactamente en el corte
        if len(keep) < len(imp_df):
            keep.append(imp_df.iloc[len(keep)]["feature"])

    else:
        if threshold is None:
            threshold = "median"
        if isinstance(threshold, str):
            thr_val = imp_df["importance"].agg(threshold)
        else:
            thr_val = float(threshold)
        keep = imp_df.loc[imp_df["importance"] >= thr_val, "feature"].tolist()

    return X[keep], keep

In [ ]:
# ===================== 3. BÚSQUEDA DEL MEJOR UMBRAL ACUMULADO =====================
def find_best_cum_threshold(
    X_train: pd.DataFrame,
    y_train: pd.Series,
    X_valid: pd.DataFrame,
    y_valid: pd.Series,
    task: str = "classification",
    thresholds: Tuple[float, ...] = (0.6, 0.7, 0.8, 0.9),
    random_state: int = 42,
    metric: str = "auto",
    **xgb_params: Any
) -> Tuple[float, pd.DataFrame, pd.DataFrame]:
    """
    Entrena un XGB en train, calcula importancias y prueba varios umbrales
    acumulados para ver cuál da la mejor métrica en valid.

    Para CLASIFICACIÓN:
        - Detecta nº de clases.
        - Métrica por defecto (metric="auto"):
            • Binaria: ROC-AUC (probabilidades de la clase positiva).
            • Multiclase: ROC-AUC macro OVR (usa predict_proba).
          Alternativas: metric="f1_macro", "accuracy", "logloss" (se MINIMIZA).
    Para REGRESIÓN:
        - Usa R^2.

    Devuelve:
        best_thr, res_df_ordenado_por_score_desc, imp_df

    Parámetros
    ----------
    X_train, y_train, X_valid, y_valid : pd.DataFrame / pd.Series
        Particiones de entrenamiento y validación.
    task : str
        "classification" (default) o "regression".
    thresholds : tuple[float, ...]
        Valores de umbral de importancia acumulada a evaluar (0-1).
    random_state : int
        Semilla para reproducibilidad.
    metric : str
        "auto" (default), "roc_auc", "f1_macro", "accuracy", "logloss" (clasif) o "r2" (regresión).
    **xgb_params : dict
        Parámetros extra para el estimador de XGBoost (pasan a compute y a los modelos internos).

    Returns
    -------
    (best_thr, res_df, imp_df)
        best_thr : float
            Umbral con mejor score (o menor logloss si metric='logloss').
        res_df : pd.DataFrame
            Tabla con resultados por umbral (n_features, score).
        imp_df : pd.DataFrame
            Importancias calculadas en X_train / y_train.
    """
    imp_df, _ = compute_xgb_importance(
        X_train, y_train, task=task, random_state=random_state, **xgb_params
    )

    results = []

    # Detectar nº de clases si es clasificación
    if task == "classification":
        classes = np.unique(y_train)
        n_classes = len(classes)
        if metric == "auto":
            metric_to_use = "roc_auc" if n_classes == 2 else "roc_auc"
        else:
            metric_to_use = metric
    else:
        metric_to_use = "r2" if metric == "auto" else metric

    for thr in thresholds:
        X_tr_sel, cols = select_features_with_importance(
            X_train, imp_df, cum_threshold=thr, top_n=None, threshold=None
        )
        X_va_sel = X_valid[cols]

        if task == "classification":
            params = dict(random_state=random_state, n_jobs=-1, tree_method="hist")
            params.update(xgb_params)

            if n_classes > 2:
                params.setdefault("objective", "multi:softprob")
                params.setdefault("num_class", n_classes)
                eval_metric = "mlogloss"
            else:
                params.setdefault("objective", "binary:logistic")
                eval_metric = "logloss"

            model_sel = XGBClassifier(eval_metric=eval_metric, **params)
            model_sel.fit(X_tr_sel, y_train)

            # Probabilidades y predicciones
            proba = model_sel.predict_proba(X_va_sel)
            pred  = np.argmax(proba, axis=1) if n_classes > 2 else (proba[:, 1] >= 0.5).astype(int)

            # Calcular métrica
            if metric_to_use == "roc_auc":
                if n_classes == 2:
                    score = roc_auc_score(y_valid, proba[:, 1])
                else:
                    # AUC macro One-vs-Rest
                    score = roc_auc_score(y_valid, proba, multi_class="ovr", average="macro")
            elif metric_to_use == "f1_macro":
                score = f1_score(y_valid, pred, average="macro")
            elif metric_to_use == "accuracy":
                score = accuracy_score(y_valid, pred)
            elif metric_to_use == "logloss":
                # En este caso, menor es mejor. Guardamos negativo para mantener criterio "mayor mejor".
                score = -log_loss(y_valid, proba, labels=np.unique(y_train))
            else:
                raise ValueError(f"Métrica no soportada: {metric_to_use}")

        else:
            # REGRESIÓN
            params = dict(random_state=random_state, n_jobs=-1, tree_method="hist")
            params.update(xgb_params)
            model_sel = XGBRegressor(**params)
            model_sel.fit(X_tr_sel, y_train)
            pred = model_sel.predict(X_va_sel)

            if metric_to_use == "r2":
                score = r2_score(y_valid, pred)
            else:
                raise ValueError(f"Métrica de regresión no soportada: {metric_to_use}")

        results.append({"cum_threshold": thr, "n_features": len(cols), "score": score})

    # Ordenar (si usamos logloss negado, mayor sigue siendo mejor)
    res_df = pd.DataFrame(results).sort_values("score", ascending=False).reset_index(drop=True)
    best_thr = float(res_df.iloc[0]["cum_threshold"])
    return best_thr, res_df, imp_df

In [ ]:
def remove_highly_correlated_features(df, threshold=0.9):

    # Solo numéricos para evitar errores y acelerar
    corr_matrix = df.corr(numeric_only=True).abs()
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape, dtype=bool), k=1))

    to_drop = []
    for col in tqdm(upper.columns, desc=f"Pruning corr > {threshold}", unit="col", leave=False):
        if (upper[col] > threshold).any():
            to_drop.append(col)

    return df.drop(columns=to_drop, errors="ignore"), to_drop


In [ ]:
# ===================== 3. PIPELINE PRINCIPAL =====================
df = df.dropna()
y = df['label']
X = df.iloc[:, 2:]

# --- 3.3 Split temporal (ejemplo simple 80/20) ---
split_idx = int(len(X) * 0.8)
X_train, X_test = X.iloc[:split_idx], X.iloc[split_idx:]
y_train, y_test = y.iloc[:split_idx], y.iloc[split_idx:]

# --- 3.4 Remove correlated features ---
X_train_filtered, dropped_features = remove_highly_correlated_features(X_train, threshold=0.9)
X_test_filtered = X_test.drop(columns=dropped_features)

# Baseline logistic regression with time-series CV
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_train_filtered)
tscv = TimeSeriesSplit(n_splits=5)
baseline = cross_val_score(LogisticRegression(max_iter=1000), X_scaled, y_train, cv=tscv).mean()
print('Logistic regression CV accuracy:', baseline)

# --- 3.5 Importancias con XGBoost ---
imp_df, xgb_model = compute_xgb_importance(X_train_filtered, y_train, task='classification')

print('=== Importancias XGBoost ===')
print(imp_df.head(20))
print(f'Total features: {len(imp_df)}')

# --- 3.6 Selección (elige una opción) ---
X_train_sel, keep_cols = select_features_with_importance(X_train_filtered, imp_df, cum_threshold=0.8)
X_test_sel = X_test_filtered[keep_cols]

print(f'Features seleccionadas: {len(keep_cols)}')
importance_map = imp_df.set_index("feature")["importance"]
selected_importances = pd.DataFrame({
    "feature": keep_cols,
    "importance": importance_map.reindex(keep_cols).values
})
selected_importances.to_csv(root_data+'Results/'+symbol+'_'+direction+'_M5M10_'+data_type+'_ImportantCols.csv', index=False)

# Save dataset with selected features
df_selected = df[['Date', 'label'] + keep_cols]
df_selected.to_csv(root_data+'Results/'+symbol+'_'+direction+'_M5M10_'+data_type+'_Features.csv', index=False)

# Time-series cross-validation with XGBoost
xgb_cv = XGBClassifier(eval_metric='logloss', n_estimators=500, max_depth=6, learning_rate=0.05, subsample=0.8, colsample_bytree=0.8, random_state=42, n_jobs=-1, tree_method='hist')
xgb_scores = cross_val_score(xgb_cv, X_train_sel, y_train, cv=tscv, scoring='accuracy')
print('XGBoost CV accuracy:', xgb_scores.mean())


Pruning corr > 0.9:   0%|          | 0/2725 [00:00<?, ?col/s]

Logistic regression CV accuracy: 0.5461760141875416
=== Importancias XGBoost ===
                                     feature  importance  cum_importance
0            M10_raw_10min_RSI_basic_14_diff    0.016185        0.016185
1                M10_scale_10min_ATR_14_diff    0.010048        0.026233
2                                 Open_Trade    0.009153        0.035386
3                 M5_scale_MACD_12_26_9_diff    0.005679        0.041065
4                        M10_raw_10min_RSI_3    0.005639        0.046704
5                      M5_raw_Kal_change_300    0.005598        0.052302
6   M10_raw_10min_slope_lin_reg_signal_900_3    0.005403        0.057705
7                  M10_raw_10min_CCI_20_diff    0.005398        0.063103
8                   M5_raw_MACD_12_26_9_diff    0.005299        0.068401
9           M10_raw_10min_slope_signal_900_3    0.005195        0.073596
10  M10_raw_10min_slope_lin_reg_signal_900_6    0.004901        0.078497
11  M10_raw_10min_slope_lin_reg_signal_600_